In [48]:
import torch

import cv2
from matplotlib import cm

import numpy as np
import os
import os.path as osp
from PIL import Image
from torchvision import transforms as T
from diffusers import AutoencoderKL

In [49]:
img = Image.open('./output_0022.jpg')
# transform = T.Compose([T.ToTensor()])
# img = np.array(img)
# img = transform(img)
# print(img[2,700:800,500:600])


In [50]:
vae = AutoencoderKL.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="vae")

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.
The config attributes {'scaling_factor': 0.18215} were passed to AutoencoderKL, but are not expected and will be ignored. Please verify your config.json configuration file.


In [51]:
# img.mode == 'RGB'
img = img.convert('RGB')

In [52]:
img = np.array(img)

In [53]:
size = 256
img = cv2.resize(img, (size, size), interpolation=cv2.INTER_CUBIC)

In [54]:
img = np.array(img).astype(np.float32)
img = img / 127.5 - 1.0
img = torch.from_numpy(img).permute(2, 0, 1)
img = torch.unsqueeze(img,0)
img.shape

torch.Size([1, 3, 256, 256])

In [55]:
latents = vae.encode(img).latent_dist.sample().detach()

In [56]:
latents.shape
# 1024*1024 （1,4,128,128）
# 512*512    (1,4,64,64)
# 256*256    (1,5,32,32)
# 128*128    (1,4,16,16)

torch.Size([1, 4, 32, 32])

In [57]:
from einops import rearrange, reduce, repeat
y = rearrange(latents, 'b c h w -> b c (h w)')
y.shape

torch.Size([1, 4, 1024])